# Identifying Potential Locations for EV Charging Stations

In this notebook, we will identify potential locations for new EV charging stations in the Kitchener-Waterloo region. We will use OpenStreetMap (OSM) data to extract points of interest (POIs) such as parking lots, gas stations, and commercial centers, which could serve as suitable locations for installing charging stations.

We will then visualize these potential locations on a map along with the existing charging stations to get a better understanding of the current charging infrastructure and the potential for expansion.

In [1]:
import osmnx as ox
import geopandas as gpd
import folium
from pathlib import Path
import sys

# Add src directory to Python path
sys.path.append(str(Path.cwd().parent / 'src'))

from visualization import map_viz
from data.api_client import OpenChargeMapClient

## Downloading POIs from OpenStreetMap

To identify potential locations for charging stations, we will download points of interest (POIs) from OpenStreetMap (OSM) for the Kitchener-Waterloo region. We will focus on the following types of POIs:

- Parking lots
- Gas stations
- Retail and commercial buildings

These types of locations are suitable for installing charging stations due to their accessibility, existing infrastructure, and proximity to popular destinations.

In [2]:
# Define the place name and tags for potential locations
place_name = "Kitchener, Ontario, Canada"
tags = {
    'amenity': ['parking', 'fuel'],
    'building': ['retail', 'commercial']
}

# Download the POIs from OpenStreetMap
pois = ox.features_from_place(place_name, tags=tags)


## Preprocessing the POI Data

After downloading the POIs from OpenStreetMap, we need to preprocess the data to extract the relevant information and convert it into a suitable format for analysis and visualization.

We will perform the following preprocessing steps:

1. Convert the POI data to a GeoDataFrame
2. Select the relevant columns: 'name', 'amenity', 'building', and 'geometry'
3. Rename the 'amenity' and 'building' columns to 'type' for consistency
4. Fill missing values in the 'type' column with 'commercial'

In [3]:
# Convert to GeoDataFrame and preprocess the data
potential_locations_gdf = gpd.GeoDataFrame(pois)
potential_locations_gdf = potential_locations_gdf[['name', 'amenity', 'building', 'geometry']]
potential_locations_gdf.loc[:, 'type'] = potential_locations_gdf[['amenity', 'building']].fillna('').apply(lambda x: x.iloc[0] if x.iloc[0] else x.iloc[1], axis=1)
potential_locations_gdf.loc[potential_locations_gdf['type'] == '', 'type'] = 'commercial'

# Display the potential locations data
potential_locations_gdf.head()

name  amenity building  \
element_type osmid                                           
node         1541858124              NaN  parking      NaN   
             1541858130              NaN  parking      NaN   
             1542510005              NaN  parking      NaN   
             1623078590  Darshan Gas Bar     fuel      NaN   
             1726088611          Pioneer     fuel      NaN   

                                           geometry     type  
element_type osmid                                            
node         1541858124  POINT (-80.49232 43.43996)  parking  
             1541858130  POINT (-80.49356 43.43985)  parking  
             1542510005  POINT (-80.49645 43.44063)  parking  
             1623078590  POINT (-80.48513 43.45789)     fuel  
             1726088611  POINT (-80.47149 43.44161)     fuel

## Visualizing Potential Locations

To visualize the potential locations for charging stations, we will create a map of the Kitchener-Waterloo region using Folium and plot the potential locations along with the existing charging stations.

This visualization will help us understand the spatial distribution of the potential locations and identify areas that may require additional charging infrastructure.

In [4]:
# Fetch existing charging stations
client = OpenChargeMapClient()
stations_df = client.fetch_stations()

# Create a base map of the KW region
m = map_viz.create_kw_map()

# Plot existing charging stations on the map
m = map_viz.plot_charging_stations(m, stations_df)

# Plot potential locations on the map
m = map_viz.plot_potential_locations(m, potential_locations_gdf)

# Cell 16: Display the map
m